In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ---------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dset_idx <- 3
save_res <- TRUE

In [3]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [4]:
matrisome_df <- rutils::load_matrisome_df(matrisome_path) %>%
    dplyr::select(gene_symbol, division, category) %>%
    dplyr::rename(geneID = gene_symbol)
demg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_demg_list.txt"))
figo_umsmg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_figo_umsmg_list.txt"))
survival_umsmg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_survival_umsmg_list.txt"))
figo_nsmg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_figo_nsmg_list.txt"))
figo_sig_list <- union(figo_umsmg_list, figo_nsmg_list)
figo_sig_de_list <- intersect(figo_sig_list, demg_list)



-- Column specification ---------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)




# Matrisome overview

In [5]:
matrisome_df %>%
    dplyr::group_by(category) %>%
    dplyr::count()

category,n
<chr>,<int>
Collagens,44
ECM Glycoproteins,195
ECM Regulators,238
ECM-affiliated Proteins,171
Proteoglycans,35
Secreted Factors,344


# FIGO UMSMG $\cap$ DEMG

In [6]:
matrisome_df %>%
    dplyr::filter(geneID %in% intersect(figo_umsmg_list, demg_list)) %>%
    dplyr::group_by(category) %>%
    dplyr::count()

category,n
<chr>,<int>
Collagens,1
ECM Glycoproteins,4
ECM Regulators,9
ECM-affiliated Proteins,1
Secreted Factors,7


In [7]:
figo_umsmg_list %>% length()
figo_sig_list %>% length()
demg_list %>% length()
figo_sig_de_list %>% length()

length(figo_sig_de_list) / length(union(figo_sig_list, demg_list))
# Use survival list as denom since it's smaller in all cases
length(intersect(survival_umsmg_list, figo_sig_list)) / length(survival_umsmg_list)

[1] 33

[1] 281

[1] 618

[1] 173

[1] 0.238292

[1] 0.3061224

# Survival UMSMG $\cap$ DEMG

In [8]:
matrisome_df %>%
    dplyr::filter(geneID %in% intersect(survival_umsmg_list, demg_list)) %>%
    dplyr::group_by(category) %>%
    dplyr::count()

category,n
<chr>,<int>
Collagens,1
ECM Glycoproteins,7
ECM Regulators,5
ECM-affiliated Proteins,5
Secreted Factors,8


# FIGO NSMG $\cap$ DEMG

In [9]:
matrisome_df %>%
    dplyr::filter(geneID %in% intersect(figo_nsmg_list, demg_list)) %>%
    dplyr::group_by(category) %>%
    dplyr::count()

category,n
<chr>,<int>
Collagens,7
ECM Glycoproteins,32
ECM Regulators,40
ECM-affiliated Proteins,35
Proteoglycans,2
Secreted Factors,43
